# Table of Contents
1. [download/read csv file](#sec1)
2. [Convert the labeled data set tags into the new individual tag columns](#sec2)
3. [NLTK STUFF LMAO](#sec3)

In [2]:
#packages
%pip install numpy
%pip install pandas
import numpy as np
import pandas as pd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


<a id = "sec1"></a>
## **Download/Read CSV file**

In [3]:
# read csv
data_path= "https://raw.githubusercontent.com/JL72005/PIT-UN-Project4/refs/heads/main/NEW_TAGS_dc_md_va_flash_floods_1996_present.xlsx%20-%20dc_md_va_flash_floods_1996_pres.csv?token=GHSAT0AAAAAADEFGP75WC7WXYUOC3DFUGL62B7SF5Q"
df = pd.read_csv(data_path)

<a id = "sec2"></a>
## **Convert the labeled data set tags into the new individual tag columns**

In [4]:
#list of tags
tag_names = ['death','injury','evacuation','rescue','car_crash','home_damage','infrastructure_damage','soft_infrastructure_damage','road_closure','power_outage','tree_damage','vehicle_loss','agricultural_damage','campground_damage']
#list of weather events
weather_names = ['nor_easter','thunderstorm','hurricane','tornado','lightning','mudslide']

In [9]:
#populate columns for OG tags
for tag in tag_names:
    df[tag] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if tag in x else 0)
for tag in weather_names:
    df[tag] = df['weather_events'].apply(str).apply(lambda x: 1 if tag in x else 0)

In [10]:
#check columns
df[tag_names+weather_names].head(15)

,death,injury,evacuation,rescue,car_crash,home_damage,infrastructure_damage,soft_infrastructure_damage,road_closure,power_outage,tree_damage,vehicle_loss,agricultural_damage,campground_damage,nor_easter,thunderstorm,hurricane,tornado,lightning,mudslide
0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
7,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0
8,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0


<a id="sec3"></a>
## **NLTK STUFF LMAO**

### Stemming

In [16]:
#imports
%pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
# TOKENIZATION & REMOVING STOP WRODS

#download necessary NLTK data
nltk.download('punkt_tab')
nltk.download('stopwords')
#download stopwords
stop_words = set(stopwords.words('english'))

def remove_stopword(text):
  tk = RegexpTokenizer(r'\w+')
  word_tokens = tk.tokenize(text)
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  filtered_text = " ".join(filtered_sentence)
  return filtered_text

df['TOKEN_EPISODE_NARRATIVE'] = df['EPISODE_NARRATIVE'].apply(str).apply(remove_stopword)
df['TOKEN_EVENT_NARRATIVE'] = df['EVENT_NARRATIVE'].apply(str).apply(remove_stopword)
df[['TOKEN_EPISODE_NARRATIVE', 'TOKEN_EVENT_NARRATIVE']].head()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\emmab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emmab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,TOKEN_EPISODE_NARRATIVE,TOKEN_EVENT_NARRATIVE
0,unusually intense squall line feeding unseason...,nan
1,Intense thunderstorms moved northeast half Was...,nan
2,rapidly developing thunderstorm knocked numero...,nan
3,Feeder bands torrential tropical rains associa...,nan
4,Several roads briefly closed torrential rains ...,nan


In [ ]:
# STEMMING

stemmer = PorterStemmer()

def stem_narrative(text):
    token_list = text.split(" ")
    stemmed_list = []
    for word in token_list:
        stemmed_list.append(stemmer.stem(word))
    return stemmed_list

df['STEM_EPISODE_NARRATIVE'] = df['TOKEN_EPISODE_NARRATIVE'].apply(str).apply(stem_narrative)
df['STEM_EVENT_NARRATIVE'] = df['TOKEN_EVENT_NARRATIVE'].apply(str).apply(stem_narrative)

df['STEM_EPISODE_NARRATIVE'].head()


0    [unusu, intens, squall, line, feed, unseason, ...
1    [intens, thunderstorm, move, northeast, half, ...
2    [rapidli, develop, thunderstorm, knock, numer,...
3    [feeder, band, torrenti, tropic, rain, associ,...
4    [sever, road, briefli, close, torrenti, rain, ...
Name: STEM_EPISODE_NARRATIVE, dtype: object